In [ ]:
import pandas as pd
import os
import re

In [ ]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Padronizacao_conversao'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [ ]:
PASTA_DE_DADOS = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_download']
PASTA_SAIDA = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_convertido']
ARQUIVO_OPERADORAS = PARAMETROS_GERAIS['arquivo_operadoras']


In [ ]:
ARQUIVO_OPERADORAS

In [ ]:
df_operadoras = pd.read_csv(ARQUIVO_OPERADORAS,dtype=str,sep="\t",encoding="ISO-8859-1")

In [ ]:
df_operadoras.head()

In [ ]:
patternPeriodo = re.compile('(.*)T(.*).csv')

In [ ]:
lista_arquivos = os.listdir(PASTA_DE_DADOS)

In [ ]:
if (not(os.path.exists(PASTA_SAIDA))):
    os.mkdir(PASTA_SAIDA)

In [ ]:
for _idx,arquivo in enumerate(lista_arquivos):
    _match = patternPeriodo.findall(arquivo)[0]
    _semestre = _match[0]
    _ano =  _match[1]
    if (int(_ano) > 2010):
        df = pd.read_csv(PASTA_DE_DADOS+arquivo,sep=";",dtype=str,encoding="ISO-8859-1")
        df = df.merge(df_operadoras,left_on='REG_ANS', right_on='Registro_ANS')
        df.insert(0,'SEMESTRE',_semestre)
        df.insert(0,'ANO',_ano)
        df.insert(0,'CHAVE',df['CD_CONTA_CONTABIL']+"_"+df['REG_ANS']+"_"+_ano+"_"+_semestre)
        df['VL_SALDO_FINAL'] = df['VL_SALDO_FINAL'].str.replace(",",".").astype(float)
        df.drop(columns="DATA",inplace=True)
        df.set_index('CHAVE',inplace=True)
        df.to_csv(PASTA_SAIDA+arquivo,encoding="ISO-8859-1")
    print("Convertido ",(_idx+1),"de",len(lista_arquivos))